In [ ]:
import os
from pathlib import Path
import time
from datetime import datetime, timedelta
import concurrent.futures
from itertools import repeat

import json
from copy import deepcopy
import collections
import numpy as np
from torch.utils.data import Dataset

from tqdm.notebook import tqdm

from functools import partial
from omegaconf import OmegaConf

from astropy.io import fits
import skimage.io as io

from ...sunscc.dataset.utils import *
from ...sunscc.dataset.transform.pipelines import Compose 
from albumentations.core.transforms_interface import DualTransform

from hydra.utils import call, instantiate

import matplotlib.pyplot as plt

%matplotlib widget
# %matplotlib inline

In [ ]:
def print_elapsed_time(st, msg):
        end_time = time.time()
        print(f'Elapsed time {msg}: {end_time - st}')

class ClassificationDatasetSuperclasses(Dataset):

    def __init__(
        self, root_dir, partition, dtypes, classes,
        first_classes, second_classes, third_classes, 
        json_file, classification='SuperClass' , transforms=None) -> None:
        super().__init__()
        if isinstance(transforms, collections.Mapping):
            transforms = partial(call, config=transforms)
        elif isinstance(transforms, collections.Sequence):
            transforms_init = []
            for transform in transforms:
                transforms_init.append(instantiate(transform))
            transforms = Compose(transforms_init)

        self.transforms = transforms
        self.root_dir = Path(root_dir)

        self.main_dtype = dtypes[0]
        self.mask_dtype = dtypes[1]

        self.json_file = self.root_dir / json_file 
        # print(self.json_file)
        self.partition_dict = None

        self.c1_mapper = {c: i for i,c in enumerate(first_classes)}
        self.c2_mapper = {c: i for i,c in enumerate(second_classes)}
        self.c3_mapper = {c: i for i,c in enumerate(third_classes)}


        with open(self.json_file, 'r') as f:
            self.partition_dict = json.load(f)[partition]

        
        assert (classification == 'Zurich') or (classification == 'McIntosh') or (classification == 'SuperClass')

        self.classification = classification

        self.FirstClass_mapper = {c: i for i,c in enumerate(first_classes)}
        self.SecondClass_mapper = {c: i for i,c in enumerate(second_classes)}
        self.ThirdClass_mapper = {c: i for i,c in enumerate(third_classes)}

        # print(classes)
        self.files = {}
        for i, bn in enumerate(sorted(list(self.partition_dict.keys()))):
            bn = bn.split('_')[0]
            # print(bn)
            cur = {}
            image_basename = bn + '.FTS'
            image_filename = self.root_dir / self.main_dtype / image_basename

            sun_mask_filename = self.root_dir / 'sun_mask' / (bn + '.png')


            mask_basename = bn + '.png'
            mask_filename = self.root_dir / self.mask_dtype / mask_basename

            conf_map_basename = bn + '_proba_map.npy'
            conf_map_filename = self.root_dir / self.mask_dtype / conf_map_basename

            cur["name"] = bn
            cur[self.main_dtype] = image_filename
            cur[self.mask_dtype] = mask_filename
            cur[self.mask_dtype+"_conf_map"] = conf_map_filename
            cur["sun_mask"] = sun_mask_filename

            self.files[bn] = cur

        self.partition_dict

        self.groups = {}
        for k,v in self.partition_dict.items():

            if v[self.classification]["1"] in classes:
                    self.groups[k] = v
            else:
                pass

        self.dataset_length = len(list(self.groups.keys()))



    def __len__(self) -> int:
        return self.dataset_length
    
    def __getitem__(self, index: int, do_transform=True):

        sample = {} # dictionnary with 'image', 'class', 'angular_excentricity', 'centroid_lat'

        k = sorted(list(self.groups.keys()))[index]
        basename = k.split('_')[0]

        group_dict = self.groups[k]

        img_name = self.files[basename][self.main_dtype] # path of FITS file
        mask_name = self.files[basename][self.mask_dtype]
        conf_map_name = self.files[basename][self.mask_dtype+"_conf_map"]

        hdulst:fits.HDUList = fits.open(img_name)
        image = hdulst[0]
        header = image.header
        center = np.array(image.shape)//2
        radius = header['SOLAR_R']
        
        sample['solar_disk'] = io.imread(self.files[basename]["sun_mask"])
        sample['excentricity_map'] = create_excentricity_map(sample['solar_disk'], radius, value_outside=-1)
        sample['mask'] = io.imread(mask_name)#.astype(float)
        sample['confidence_map'] = np.load(conf_map_name)

        sample['image'] = (image.data).astype(float)

        sample['members'] = np.array(group_dict['members']) if 'members' in group_dict else np.array([0])
        sample['members_mean_px'] = np.array(group_dict['members_mean_px']) if 'members_mean_px' in group_dict else np.array([0])

        sample['name'] = basename
        sample['group_name'] = k

        sample['solar_angle'] = group_dict['angle']
        sample['deltashapeX'] = group_dict['deltashapeX']
        sample['deltashapeY'] = group_dict['deltashapeY']
        
        sample['angular_excentricity'] = np.array([group_dict["angular_excentricity_deg"]])
        sample['centroid_px'] = np.array(group_dict["centroid_px"])
        sample['centroid_Lat'] = np.array([group_dict["centroid_Lat"]])

        sample['class1'] = group_dict[self.classification]['1']
        sample['class2'] = group_dict[self.classification]['2']
        sample['class3'] = group_dict[self.classification]['3']
        
        if sample["image"].shape == (1024,1024):
            fig,ax = plt.subplots(2, 5, figsize=(10, 4) )
            ax[0,0].imshow(sample["image"], cmap='gray', interpolation='none')
            ax[0,1].imshow(sample["mask"], cmap='gray', interpolation='none')
            ax[0,2].imshow(sample["confidence_map"], cmap='gray', interpolation='none')
            ax[0,3].imshow(sample["solar_disk"], cmap='gray', interpolation='none')
            ax[0,4].imshow(sample["excentricity_map"], cmap='gray', interpolation='none')
        
            ax[0,0].scatter(sample['centroid_px'][0], sample['centroid_px'][1], c='r', s=10)

            sample["image"] = np.repeat(np.repeat(sample["image"], 2, axis=0), 2, axis=1)
            sample["mask"] = np.repeat(np.repeat(sample["mask"], 2, axis=0), 2, axis=1)
            sample["confidence_map"] = np.repeat(np.repeat(sample["confidence_map"], 2, axis=0), 2, axis=1)
            sample["solar_disk"] = np.repeat(np.repeat(sample["solar_disk"], 2, axis=0), 2, axis=1)
            sample["excentricity_map"] = np.repeat(np.repeat(sample["excentricity_map"], 2, axis=0), 2, axis=1)
        
            sample['deltashapeX'] = sample['deltashapeX']*2
            sample['deltashapeY'] = sample['deltashapeY']*2

            # also double the centroid values
            sample['centroid_px'] = sample['centroid_px']*2


            ax[1,0].imshow(sample["image"], cmap='gray', interpolation='none')
            ax[1,1].imshow(sample["mask"], cmap='gray', interpolation='none')
            ax[1,2].imshow(sample["confidence_map"], cmap='gray', interpolation='none')
            ax[1,3].imshow(sample["solar_disk"], cmap='gray', interpolation='none')
            ax[1,4].imshow(sample["excentricity_map"], cmap='gray', interpolation='none')
            # scatter the centroid
            ax[1,0].scatter(sample['centroid_px'][0], sample['centroid_px'][1], c='r', s=10)

            fig.tight_layout()
            
        flip_time = "2003-03-08T00:00:00"
        date = whitelight_to_datetime(basename)
        datetime_str = datetime_to_db_string(date).replace(' ', 'T')
        
        should_flip = (datetime.fromisoformat(datetime_str) - datetime.fromisoformat(flip_time)) < timedelta(0)
        sample['should_flip'] = should_flip

        if should_flip:
            sample['image'] = np.flip(sample['image'],axis=0)
            sample['solar_disk'] = np.flip(sample['solar_disk'],axis=0)
            sample['mask'] = np.flip(sample['mask'],axis=0)
            sample['confidence_map'] = np.flip(sample['confidence_map'],axis=0)
            sample['excentricity_map'] = np.flip(sample['excentricity_map'],axis=0)

        if self.transforms is not None and do_transform:
            sample = self.transforms(**sample)
        
        return sample
        

In [ ]:
root_dir = "../datasets/Classification_dataset/2002-2019"
partition = 'train'
dtypes = ['image', 'T425-T375-T325_fgbg']

classes = ['A','B','C','SuperGroup','H']
first_classes = [ 'A','B','C','SuperGroup','H']
second_classes= [ 'x','r','sym','asym']
third_classes= [ "x","o","frag"]
json_file = 'test/dataset_final.json'
classification = 'SuperClass'
transforms = OmegaConf.load('../sunscc/conf/exp/Classification_Superclasses4.yaml').dataset.train_dataset.transforms

transforms[3].standard_height = 350
transforms[3].standard_width = 350

dataset =  ClassificationDatasetSuperclasses(root_dir, partition, dtypes, classes, first_classes, second_classes, third_classes, json_file, classification, transforms)

In [ ]:
partition_samples = {}

In [ ]:
def get_sample(idx, dataset):
    tmp = dataset[idx]
    center  = tmp['image'].shape[0]//2 , tmp['image'].shape[1]//2
    slice_x = center[0]-( 256 //2 ), center[0]+ (256 //2 )
    slice_y = center[1]-( 256 //2 ), center[1]+ (256 //2 )
    center_region = tmp['confidence_map'][slice_x[0]:slice_x[1], slice_y[0]:slice_y[1]]
    if np.sum(center_region) == 0:
        print('skipping sample', idx)
        return None
    return dataset[idx]

num_cpu = 15

with concurrent.futures.ProcessPoolExecutor(max_workers=int(num_cpu)) as executor:
    for sample in tqdm(executor.map(get_sample, range(len(dataset)) , repeat(deepcopy(dataset)))):
    # for sample in tqdm(executor.map(get_sample, range(10) , repeat(deepcopy(dataset)))):
        if sample is not None:
            if sample['group_name'] not in partition_samples:
                partition_samples[sample['group_name']] = sample


In [ ]:
npy_file = os.path.join(root_dir, 'test', f'all_samples_{partition}.npy')

np.save(npy_file, partition_samples)

# Join the train, val, test .npy files


In [ ]:
all_samples ={'train':{},'val':{},'test':{}}
for p in all_samples.keys():
    print('loading', p)
    st = time.time()    
    filename = os.path.join(root_dir,'test',f'all_samples_{p}.npy' )
    tmp = np.load(filename, allow_pickle=True).item()
    print('Elapsed time', time.time()-st)
    
    all_samples[p] = tmp

In [ ]:
print("Dumping")
st = time.time()    
tot_npy_file = os.path.join(root_dir, 'test', f'all_samples.npy')
np.save(tot_npy_file, all_samples)
print('Elapsed time', time.time()-st)

# Test

In [ ]:
def print_elapsed_time(st, msg):
        end_time = time.time()
        print(f'Elapsed time {msg}: {end_time - st}')
        
class Deepsun_Focus_Move(DualTransform):
    def __init__(self, standard_height=256, standard_width=256,
                        focus_on_group=True,
                        random_move=False, random_move_percent=0.1,  
                        always_apply=False, p=1.0) -> None:

        super().__init__(always_apply, p)

        self.standard_height = standard_height
        self.standard_width = standard_width
        
        self.focus_on_group = focus_on_group
        self.random_move = random_move
        self.random_move_percent = random_move_percent

    def get_bounding_box_around_group_with_padding(self, mask, offset):
        # Get the bounding box around non-zero pixels in mask
        x, y = np.nonzero(mask)
        # print(x, y)
        x1, x2 = (np.min(x), np.max(x)) if len(x) > 0 else (None, None)
        y1, y2 = (np.min(y), np.max(y)) if len(y) > 0 else (None, None)

        if (x1 is None) or (y1 is None):
            return 0, mask.shape[0]-1, 0, mask.shape[1]-1

        # Add padding
        x1 -= offset
        x2 += offset
        y1 -= offset
        y2 += offset


        # Make sure the bounding box is not outside the image
        x1 = max(x1, 0)
        x2 = min(x2, mask.shape[0])
        y1 = max(y1, 0)
        y2 = min(y2, mask.shape[1])

        return x1, x2, y1, y2

    def adapt_bbox_to_image_size(self, bbox, image_size):
        bbox_center = ((bbox[0] + bbox[1]) // 2, (bbox[2] + bbox[3]) // 2)
        bbox_size = (bbox[1] - bbox[0], bbox[3] - bbox[2])

        # if bbox is too small, expand it
        minimal_percentage = .4

        bbox_size = (max(bbox_size[0], image_size[0] * minimal_percentage),
                     max(bbox_size[1], image_size[1] * minimal_percentage))
        
        return (int(bbox_center[0] - bbox_size[0] // 2), int(bbox_center[0] + bbox_size[0] // 2),
                int(bbox_center[1] - bbox_size[1] // 2), int(bbox_center[1] + bbox_size[1] // 2))

    def crop_img(self, img, bbox):
        # Crop image
        x1, x2, y1, y2 = bbox
        img = img[x1:x2, y1:y2]
        return img
        
    def padding(self, array, xx, yy):
        """
        :param array: numpy array
        :param xx: desired height
        :param yy: desirex width
        :return: padded array
        """

        h = array.shape[0]
        w = array.shape[1]

        a = (xx - h) // 2
        aa = xx - a - h

        b = (yy - w) // 2
        bb = yy - b - w

        a = max(a,0)
        b = max(b,0)
        aa = max(aa,0)
        bb = max(bb,0)

        return np.pad(array, pad_width=((a, aa), (b, bb)), mode='constant')

    def crop_and_pad(self, img, bbox, image_size):
        
        # Crop image
        img = self.crop_img(img, bbox)
        # Pad image
        img = self.padding(img, image_size[0], image_size[1])
        return img

    def data_aug_random_move(self, bbox, max_offset):
        '''
        Randomly move the bounding box
        param bbox: bounding box
        param max_offset: maximum offset in portion of the bbox size
        '''
        # Randomly move the bounding box
        x1, x2, y1, y2 = bbox
        horizontal_offset = (np.random.random(1) * 2*max_offset) - max_offset
        vertical_offset = (np.random.random(1) * 2*max_offset) - max_offset
        
        x1 += int(horizontal_offset * (bbox[1] - bbox[0]))
        x2 += int(horizontal_offset * (bbox[1] - bbox[0]))
        y1 += int(vertical_offset * (bbox[3] - bbox[2]))
        y2 += int(vertical_offset * (bbox[3] - bbox[2]))
        
        return x1, x2, y1, y2
        
    def __call__(self, *args, force_apply=False, **kwargs):

        img_group_crop = kwargs['image'].copy()
        msk_group_crop = kwargs['mask'].copy()
        grp_msk_group_crop = kwargs['group_mask'].copy()
        disk_group_crop = kwargs['solar_disk'].copy()
        excentricity_group_crop = kwargs['excentricity_map'].copy()
        confidence_group_crop = kwargs['confidence_map'].copy()
        grp_confidence_group_crop = kwargs['group_confidence_map'].copy()
        
        shape  = img_group_crop.shape
        
        bbox = self.get_bounding_box_around_group_with_padding((grp_confidence_group_crop>0), 10)

        
        minX, maxX, minY, maxY =  (
                                    ((bbox[1]+bbox[0])//2)-(self.standard_height//2), 
                                    ((bbox[1]+bbox[0])//2)+(self.standard_height//2), 
                                    ((bbox[3]+bbox[2])//2)-(self.standard_width//2), 
                                    ((bbox[3]+bbox[2])//2)+(self.standard_width//2)
                                    
                                    )
        
            
        if self.focus_on_group:
            # focus on the group
            # Modify the bounding box if data augmentation is enabled
            if self.random_move:
                # print('random_move')
                bbox = self.data_aug_random_move(bbox, max_offset=self.random_move_percent)
                                    
                # Make sure the bounding box is not outside the image
                x1, x2, y1, y2 = bbox
                x1 = max(x1, 0)
                x2 = min(x2, self.standard_width)
                y1 = max(y1, 0)
                y2 = min(y2, self.standard_height)
                bbox = x1, x2, y1, y2
            else:
               
                pass
                
            bbox = self.adapt_bbox_to_image_size( bbox, (self.standard_height, self.standard_width))
            img_group_crop = self.crop_and_pad(img_group_crop, bbox, (self.standard_height, self.standard_width))
            msk_group_crop = self.crop_and_pad(msk_group_crop, bbox, (self.standard_height, self.standard_width))
            grp_msk_group_crop = self.crop_and_pad(grp_msk_group_crop, bbox, (self.standard_height, self.standard_width))
            disk_group_crop = self.crop_and_pad(disk_group_crop, bbox, (self.standard_height, self.standard_width))
            excentricity_group_crop = self.crop_and_pad(excentricity_group_crop, bbox, (self.standard_height, self.standard_width))
            confidence_group_crop = self.crop_and_pad(confidence_group_crop, bbox, (self.standard_height, self.standard_width))
            grp_confidence_group_crop = self.crop_and_pad(grp_confidence_group_crop, bbox, (self.standard_height, self.standard_width))
        else:
            if self.random_move:
                
                frac = np.max([(bbox[1]-bbox[0]) /self.standard_height, (bbox[3]-bbox[2]) /self.standard_width])
                frac = np.sqrt(frac)
                
                bbox = self.data_aug_random_move([minX,maxX,minY,maxY], max_offset=self.random_move_percent*frac)
                
                if not ((bbox[1] > shape[0]) or (bbox[3] > shape[1]) or (bbox[0] < 0) or (bbox[2] < 0)):
                    bbox = [bbox[0], bbox[0]+self.standard_height, bbox[2], bbox[3]]
                    minX,maxX,minY,maxY = bbox
                
            else:
                
                pass
                
            img_group_crop = img_group_crop[minX:maxX,minY:maxY]
            msk_group_crop = msk_group_crop[minX:maxX,minY:maxY]
            grp_msk_group_crop = grp_msk_group_crop[minX:maxX,minY:maxY]
            disk_group_crop = disk_group_crop[minX:maxX,minY:maxY]
            excentricity_group_crop = excentricity_group_crop[minX:maxX,minY:maxY]
            confidence_group_crop = confidence_group_crop[minX:maxX,minY:maxY]
            grp_confidence_group_crop = grp_confidence_group_crop[minX:maxX,minY:maxY]

        if img_group_crop.shape != (self.standard_height, self.standard_width):
            img_group_crop = self.padding(img_group_crop, self.standard_height, self.standard_width)
            msk_group_crop = self.padding(msk_group_crop, self.standard_height, self.standard_width)
            grp_msk_group_crop = self.padding(grp_msk_group_crop, self.standard_height, self.standard_width)
            disk_group_crop = self.padding(disk_group_crop, self.standard_height, self.standard_width)
            excentricity_group_crop = self.padding(excentricity_group_crop, self.standard_height, self.standard_width)
            confidence_group_crop = self.padding(confidence_group_crop, self.standard_height, self.standard_width)
            grp_confidence_group_crop = self.padding(grp_confidence_group_crop, self.standard_height, self.standard_width)


        assert img_group_crop.shape == (self.standard_height, self.standard_width)
        assert msk_group_crop.shape == (self.standard_height, self.standard_width)
        assert grp_msk_group_crop.shape == (self.standard_height, self.standard_width)
        assert disk_group_crop.shape == (self.standard_height, self.standard_width)
        assert excentricity_group_crop.shape == (self.standard_height, self.standard_width)
        assert confidence_group_crop.shape == (self.standard_height, self.standard_width)
        assert grp_confidence_group_crop.shape == (self.standard_height, self.standard_width)

        kwargs['image'] = img_group_crop.copy()
        kwargs['mask'] = msk_group_crop.copy()
        kwargs['group_mask'] = grp_msk_group_crop.copy()
        kwargs['solar_disk'] = disk_group_crop.copy()
        kwargs['excentricity_map'] = excentricity_group_crop.copy()
        kwargs['confidence_map'] = confidence_group_crop.copy()
        kwargs['group_confidence_map'] = grp_confidence_group_crop.copy()
        
        return kwargs

 


class ClassificationDatasetSuperclasses_fast(Dataset):

    def __init__(
        self, root_dir, partition, dtypes, classes,
        first_classes, second_classes, third_classes, 
        dataset_file, classification='SuperClass' , transforms=None) -> None:
        super().__init__()
        if isinstance(transforms, collections.Mapping):
            transforms = partial(call, config=transforms)
        elif isinstance(transforms, collections.Sequence):
            transforms_init = []
            for transform in transforms:
                transforms_init.append(instantiate(transform))
            transforms = Compose(transforms_init)

        self.transforms = transforms
        self.root_dir = Path(root_dir)

        self.main_dtype = dtypes[0]
        self.mask_dtype = dtypes[1]


        self.disk_file = self.root_dir / dataset_file.replace('.', '_'+partition+'.')

        self.partition_dict = None

        self.c1_mapper = {c: i for i,c in enumerate(first_classes)}
        self.c2_mapper = {c: i for i,c in enumerate(second_classes)}
        self.c3_mapper = {c: i for i,c in enumerate(third_classes)}
        
        st = time.time()
        
        dataset = np.load(self.disk_file, allow_pickle=True).item()
        
        self.partition_dict = dataset

        self.classification = classification

        self.FirstClass_mapper = {c: i for i,c in enumerate(first_classes)}
        self.SecondClass_mapper = {c: i for i,c in enumerate(second_classes)}
        self.ThirdClass_mapper = {c: i for i,c in enumerate(third_classes)}

        self.groups = {}
        for k,v in self.partition_dict.items():

            if v["class1"] in classes:
                    self.groups[k] = v
            else:
                pass

        self.dataset_length = len(list(self.groups.keys()))


    def __len__(self) -> int:
        return self.dataset_length
    
    def __getitem__(self, index: int, do_transform=True):

        idx = list(self.groups.keys())[index]
        
        sample = deepcopy(self.groups[idx])


        sample['class1'] = np.array([self.FirstClass_mapper[sample['class1']]])
        sample['class2'] = np.array([self.SecondClass_mapper[sample['class2']]])
        sample['class3'] = np.array([self.ThirdClass_mapper[sample['class3']]])

       
        fig,ax = plt.subplots(2, 5, figsize=(10, 4) )
        ax[0,0].set_title(sample['group_name'])
        ax[0,0].imshow(sample["image"], cmap='gray', interpolation='none')
        ax[0,1].imshow(sample["mask"], cmap='gray', interpolation='none')
        ax[0,2].imshow(sample["confidence_map"], cmap='gray', interpolation='none')
        ax[0,3].imshow(sample["solar_disk"], cmap='gray', interpolation='none')
        ax[0,4].imshow(sample["excentricity_map"], cmap='gray', interpolation='none')


        tmp = np.argwhere(sample['confidence_map'] > 0)
        ax[0,2].scatter(tmp[:,1], tmp[:,0], s=1, c='r', alpha=0.5)

        sample['image'][sample['excentricity_map'] < 0] = 0
        sample['mask'][sample['excentricity_map'] < 0] = 0
        sample['group_mask'][sample['excentricity_map'] < 0] = 0
        sample['solar_disk'][sample['excentricity_map'] < 0] = 0
        sample['confidence_map'][sample['excentricity_map'] < 0] = 0
        sample['group_confidence_map'][sample['excentricity_map'] < 0] = 0
        sample['group_confidence_map'][sample['excentricity_map'] > 0.95] = 0
        
        sample['excentricity_map'][sample['excentricity_map'] < 0] = 0
        
        # st = time.time()
        if self.transforms is not None and do_transform:
            sample = self.transforms(**sample)
        # print_elapsed_time(st, 'transform')

        

        ax[1,0].imshow(sample["image"], cmap='gray', interpolation='none')
        ax[1,1].imshow(sample["mask"], cmap='gray', interpolation='none')
        ax[1,2].imshow(sample["confidence_map"], cmap='gray', interpolation='none')
        ax[1,3].imshow(sample["solar_disk"], cmap='gray', interpolation='none')
        ax[1,4].imshow(sample["excentricity_map"], cmap='gray', interpolation='none')
        
        fig.tight_layout()

        tmp = np.argwhere(sample['confidence_map'] > 0)
        ax[1,2].scatter(tmp[:,1], tmp[:,0], s=1, c='r', alpha=0.5)

        return sample
        

In [ ]:

root_dir = "../dataset/Classification_dataset/2002-2019"
partition='train'
dtypes = ['image', 'T425-T375-T325_fgbg']

classes = ['A','B','C','SuperGroup','H']
first_classes = [ 'A','B','C','SuperGroup','H']
second_classes= [ 'x','r','sym','asym']
third_classes= [ "x","o","frag"]
classification='SuperClasses'

transforms2 = [{'_target_': Deepsun_Focus_Move, 'standard_height': 256, 'standard_width': 256,
                 'focus_on_group': False, 'random_move': False, 'random_move_percent': .2}]
dataset_file = os.path.join(root_dir, 'test', f'all_samples.npy')
d2 = ClassificationDatasetSuperclasses_fast(root_dir, partition, dtypes, classes, first_classes, second_classes, third_classes, 
                                                dataset_file, classification, transforms2)


In [ ]:
for i in tqdm(range(len(d2))):
    try:
        tmp = d2[i]
    except:
        print(tmp['image'].shape)
        print(i)
        pass

In [ ]:
for i in range(10):
    d2[i]
pass